In [13]:
import pmdarima as pm
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np
import datetime
import requests
import warnings

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
from sklearn.preprocessing import StandardScaler
# from fbprophet import Prophet
# from fbprophet.plot import plot_plotly, add_changepoints_to_plot
# from keras.models import Sequential
# from keras.layers import Dense

# from keras.optimizers import Adam

warnings.filterwarnings('ignore')

In [14]:
czech  = pd.read_csv('univariate_models/univariate_df_Czechia.csv')
eston  = pd.read_csv('univariate_models/univariate_df_Estonia.csv')
franc  = pd.read_csv('univariate_models/univariate_df_France.csv')
malay  = pd.read_csv('univariate_models/univariate_df_Malaysia.csv')
belgi  = pd.read_csv('univariate_models/univariate_df_Belgium.csv')
chile  = pd.read_csv('univariate_models/univariate_df_Chile.csv')
irela  = pd.read_csv('univariate_models/univariate_df_Ireland.csv')
israe  = pd.read_csv('univariate_models/univariate_df_Israel.csv')
italy  = pd.read_csv('univariate_models/univariate_df_Italy.csv')
us  = pd.read_csv('univariate_models/univariate_df_United_States.csv')

# setting index to the date time value 
def index_dt(c):
    c.set_index('date')

for x in [czech, eston, franc, malay,belgi, chile, irela, israe, italy, us]:
    x = index_dt(x)  
    

In [15]:
#sort index
for x in [czech, eston, franc, malay,belgi, chile, irela, israe, italy, us]:
    x = x.sort_index()
train_size = int(len(czech) * 0.8)
czech_train, czech_test = czech[0:train_size], czech[train_size:]
eston_train, eston_test = eston[0:train_size], eston[train_size:]
franc_train, franc_test = franc[0:train_size], franc[train_size:]
malay_train, malay_test = malay[0:train_size], malay[train_size:]
belgi_train, belgi_test = belgi[0:train_size], belgi[train_size:]
chile_train, chile_test = chile[0:train_size], chile[train_size:]
irela_train, irela_test = irela[0:train_size], irela[train_size:]
israe_train, israe_test = israe[0:train_size], israe[train_size:]
italy_train, italy_test = italy[0:train_size], italy[train_size:]
us_train, us_test = us[0:train_size], us[train_size:]
us_test

,Unnamed: 0,date,new_deaths,diff1
161,261092,2023-04-02,1746.0,-338.0
162,261099,2023-04-09,1729.0,-17.0
163,261106,2023-04-16,1290.0,-439.0
164,261113,2023-04-23,1198.0,-92.0
165,261120,2023-04-30,1046.0,-152.0
166,261127,2023-05-07,1146.0,100.0
167,261134,2023-05-14,1943.0,797.0
168,261141,2023-05-21,755.0,-1188.0
169,261148,2023-05-28,738.0,-17.0
170,261155,2023-06-04,689.0,-49.0


In [16]:
from pmdarima import auto_arima
auto_model = auto_arima(czech_train['new_deaths'], seasonal=True, m=12, suppress_warnings=True, stepwise=True)

print(auto_model.summary())
n_periods=12
forecast, conf_int = auto_model.predict(n_periods=n_periods, return_conf_int=True)

# Print the forecast and confidence interval
print("Forecast:", forecast)
print("Confidence Interval:", conf_int)

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  161
Model:               SARIMAX(2, 1, 1)   Log Likelihood                -900.684
Date:                Thu, 22 Feb 2024   AIC                           1809.368
Time:                        16:49:39   BIC                           1821.669
Sample:                             0   HQIC                          1814.363
                                - 161                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          1.3489      0.092     14.625      0.000       1.168       1.530
ar.L2         -0.5973      0.058    -10.273      0.000      -0.711      -0.483
ma.L1         -0.5726      0.115     -4.969      0.0

In [17]:
cforecast_index = pd.date_range(czech_test.index[-1], periods=n_periods + 1, freq=czech_test.index.freq)[-n_periods:]

plt.plot(czech_test.index, czech_test['new_deaths'], label='Actual')

plt.plot(cforecast_index, forecast, label='Forecast', color='orange')

plt.fill_between(cforecast_index, conf_int[:, 0], conf_int[:, 1], color='orange', alpha=0.2, label='Confidence Interval')

# Customize the plot
plt.title('AutoARIMA Forecast with Confidence Interval')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.show()

AttributeError: 'RangeIndex' object has no attribute 'freq'